In [ ]:
import pandas as pd
from iminuit import Minuit 
from iminuit.cost import LeastSquares
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2

import os
while not os.getcwd().endswith('Laboratorio_SiPM'):
   os.chdir('..')
 
os.getcwd()
#import sys
#sys.path.append('Config')
#from lablib import *'

In [ ]:
def read (file_path):
    return(pd.read_csv(file_path, delim_whitespace= True, skiprows = 65, header=None, encoding= 'ISO-8859-1'))

In [ ]:
def cut_df(df: pd.DataFrame, sec: tuple) -> pd.DataFrame:
    """
    Rimuove i dati al di fuori del range definito da sec.
    """
    return df[(df.iloc[:,0] >= sec[0]) & (df.iloc[:,0] <= sec[1])].copy()


In [ ]:
def gaussian(X, S, Z, K):
    'K = fattore di scala'
    return(K*(1/(np.sqrt(2*np.pi)*S)*np.exp(-((X-Z)/S)**2)))

In [ ]:
def func_Na (X, S, Z, K, a, c):
    'K e D = fattori di scala'
    return gaussian(X, S, Z, K) + a*X**(-1) + c
'plot per il sodio'

In [ ]:
B = read('Dati\Parte 2\LySO\Bkg_30min.txt')
C = read('Dati\Parte 2\LySO\Cobalto_30min.txt')
S = read('Dati\Parte 2\LySO\Sodio_30min.txt')

C_spuro = cut_df(C, (1900, 4500)) 
B_spuro1 = cut_df(B, (1900, 4500))

St = np.sqrt(C_spuro[1] + B_spuro1[1])
C_puro = C_spuro[1] - B_spuro1[1]


fig, ax = plt.subplots(nrows = 1, ncols = 1)  
plt.plot(B_spuro1[0], C_puro, label='Cobalto 55.5V', color='blue')
ax.errorbar(B_spuro1[0], C_puro, yerr=St, fmt='o', color='red', label='Cobalto 55.5V')



In [ ]:
def analisi_Cobalto(X, Y, St):

    L_S = LeastSquares(X, Y, St, gaussian)
    my_minuit = Minuit (L_S, S = 100, K = 3000, Z = 2500)  
    my_minuit.migrad () 
    my_minuit.hesse ()   

#validità
    V = my_minuit.valid
    print('Validità: ', V)
    Q_squared = my_minuit.fval
    print ('Q-squared: ', Q_squared)
    N_dof = my_minuit.ndof
    print ('DOF: ', N_dof)
    p_v = 1 - chi2.cdf(Q_squared, N_dof)
    print('P value: ', p_v )

#visualizzazione 
    display(my_minuit)

#valori dell'interpolazione
    a_f = my_minuit.values[0]
    b_f = my_minuit.values[1]
    c_f = my_minuit.values[2]

    print('Deviazione standard: ', a_f)
    print('Fattore di scala: ', c_f)
    print('Posizione picco in ADC counts: ', b_f)


Z = analisi_Cobalto(C_spuro[0], C_puro, St)

In [ ]:
B = read('Dati\Parte 2\LySO\Bkg_30min.txt')
S = read('Dati\Parte 2\LySO\Sodio_30min.txt')

S_spuro = cut_df(S, (5000, 15000)) 
B_spuro2 = cut_df(B, (5000, 15000))

Stat = np.sqrt(S_spuro[1] + B_spuro2[1])
S_puro = S_spuro[1] - B_spuro2[1]

fig, ax  = plt.subplots(nrows = 1, ncols = 1)
plt.plot(B_spuro2[0], S_puro, label='Cobalto 55.5V', color='blue')
ax.errorbar(B_spuro2[0], S_puro, yerr=Stat, fmt='o', color='red', label='Cobalto 55.5V')



In [ ]:
def analisi_Sodio(X, Y, St):

    L_S = LeastSquares(X, Y, St, func_Na)
    my_minuit = Minuit (L_S, S = 100, K = 4000, Z = 10000, a = 0, c = 0)  
    my_minuit.migrad () 
    my_minuit.hesse ()   

#validità
    V = my_minuit.valid
    print('Validità: ', V)
    Q_squared = my_minuit.fval
    print ('Q-squared: ', Q_squared)
    N_dof = my_minuit.ndof
    print ('DOF: ', N_dof)
    p_v = 1 - chi2.cdf(Q_squared, N_dof)
    print('P value: ', p_v )

#visualizzazione 
    display(my_minuit)

#valori dell'interpolazione
    a_f = my_minuit.values[0]
    b_f = my_minuit.values[1]
    c_f = my_minuit.values[2]
    d_f = my_minuit.values[3]
    e_f = my_minuit.values[4]

    print('Deviazione standard: ', a_f)
    print('Fattore di scala: ', c_f)
    print('Posizione picco in ADC counts: ', b_f)
    print('Valori del fondo iperbolico, a :', d_f, 'c :', e_f)


Z = analisi_Sodio(S_spuro[0], S_puro, Stat)